# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine all csv files in directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
all_data

# 438000  rows, 73 columns

,Scenario,Date/Time,LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Daylight Illuminance Setpoint Exceeded Time [hr](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Glare Index [](Hourly),LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Glare Index Setpoint Exceeded Time [hr](Hourly),WINDOW_LDF1_1_BOT.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_LDF1_1_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_LDF1_1_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_LDF1_1_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),...,WINDOW_SDL2_1_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_1_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_1_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_BOT.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 Illuminance [lux](Hourly),WINDOW_SDL2_2_TOP.UNIT1:Daylighting Window Reference Point 1 View Luminance [cd/m2](Hourly),SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly),MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly),DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly)
0,Clear/Clear,01/01 01:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,612369.8311,1.862867e+07,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,669015.0221,2.035185e+07,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,716354.4116,2.179194e+07,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761760.3396,2.317322e+07,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,783507.2899,2.383478e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000e+00,0.0
8756,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000e+00,0.0
8757,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12274.9767,3.734124e+05,0.0
8758,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153771.6998,4.677830e+06,0.0


In [5]:
# all_data.dtypes

In [6]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [7]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [8]:
all_data1 = all_data1.dropna().reset_index(drop=True)

# all_data1

#438000  rows, 73 columns

-----

## Get just the wanted data (columns)

In [9]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get annual energy data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [10]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [11]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [12]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get annual cost data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variable needs to be defined above

In [13]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [14]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [15]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine annual energy and cost data
Combine two dfs<br>
Group by scenario <br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs, how="inner" )

#allAnnualData

In [17]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

allAnnualData

,Scenario,Date_Time,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
0,Clear/Clear,01/01 01:00:00,0.580415,17.656572,0.0,0.019052,0.195988,0.0,18.236986,0.215039
1,Clear/Clear,01/01 02:00:00,0.634104,19.289833,0.0,0.020814,0.214117,0.0,19.923937,0.234931
2,Clear/Clear,01/01 03:00:00,0.678973,20.654778,0.0,0.022287,0.229268,0.0,21.333751,0.251555
3,Clear/Clear,01/01 04:00:00,0.722009,21.963976,0.0,0.023699,0.243800,0.0,22.685985,0.267499
4,Clear/Clear,01/01 05:00:00,0.742622,22.591009,0.0,0.024376,0.250760,0.0,23.333631,0.275136
...,...,...,...,...,...,...,...,...,...,...
455515,lowE(2)/Clear-BBG75,12/31 20:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
455516,lowE(2)/Clear-BBG75,12/31 21:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
455517,lowE(2)/Clear-BBG75,12/31 22:00:00,0.011634,0.353927,0.0,0.000382,0.003929,0.0,0.365561,0.004310
455518,lowE(2)/Clear-BBG75,12/31 23:00:00,0.145747,4.433728,0.0,0.004784,0.049214,0.0,4.579475,0.053998


---
## Get scenario names 
Get the names to a list<br>
Export to file

In [18]:
# # Create new df of just scenario names
# # Export as a csv for javascript filter
# scenarios = allAnnualData["Scenario"].unique()
# print(scenarios)

In [19]:
# scenarios.tofile("data/final/hou_scenarios.csv", sep=",", format="%s")

---
## Group and export
Group dataframe by scenario<br>
Export to file

In [20]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"])

annualData.sum().round(2)

,FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($)
Scenario,,,,,,,,
Clear/Clear,6010.00,19906.00,33950.04,197.27,220.96,1114.37,59866.04,1532.60
Clear/Clear-BBG-15,5377.44,22079.26,29665.12,176.51,245.08,973.73,57121.82,1395.32
Clear/Clear-BBG-30,5394.21,21412.56,29882.44,177.06,237.68,980.86,56689.20,1395.60
Clear/Clear-BBG-45,5282.05,21149.57,29236.82,173.38,234.76,959.67,55668.44,1367.81
Clear/Clear-BBG-60,5009.88,21366.82,27523.95,164.44,237.17,903.45,53900.66,1305.06
Clear/Clear-BBG-75,4615.65,22000.47,24961.45,151.50,244.21,819.33,51577.58,1215.04
Clear/Clear-BBG0,5280.23,22746.31,28948.01,173.32,252.48,950.19,56974.55,1375.99
Clear/Clear-BBG15,5153.71,23031.01,28115.28,169.17,255.64,922.85,56299.99,1347.66
Clear/Clear-BBG30,4996.76,22982.69,27153.03,164.01,255.11,891.27,55132.48,1310.39


In [21]:
# Export grouped file to csv
annualData.sum().round(2).to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get daily and monthly energy data
Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh

In [22]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [23]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [24]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh)
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0
...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0


In [25]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get monthly cost data

Group by scenario <br>
Groups by month <br>
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [26]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [27]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [28]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly_costs

,Scenario,Date_Time,Illuminance(lux),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.0,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.000382,0.003929,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.004784,0.049214,0.0


In [29]:
# get_monthly_costs["Month"] = get_monthly_costs["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly_costs

---

## Combine monthly energy and cost data
Combine two dfs<br>
Create Month Column<br>
Group by scenario <br>
Export combine file

In [30]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [31]:
# Add energy and cost totals
totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [32]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyEnergy(kBtu),MonthlyCost($),Month
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.346668,0.215039,01
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.378736,0.234931,01
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.405535,0.251555,01
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.431240,0.267499,01
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.443551,0.275136,01
...,...,...,...,...,...,...,...,...,...,...,...,...
446157,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,12
446158,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,12
446159,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.006949,0.004310,12
446160,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.087052,0.053998,12


In [33]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)", "MonthlyEnergy(kBtu)" ], axis=1)
allMonthlyData2

,Scenario,Date_Time,FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyCost($),Month
0,Clear/Clear,01/01 01:00:00,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.215039,01
1,Clear/Clear,01/01 02:00:00,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.234931,01
2,Clear/Clear,01/01 03:00:00,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.251555,01
3,Clear/Clear,01/01 04:00:00,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267499,01
4,Clear/Clear,01/01 05:00:00,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136,01
...,...,...,...,...,...,...,...,...,...,...
446157,lowE(2)/Clear-BBG75,12/31 20:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
446158,lowE(2)/Clear-BBG75,12/31 21:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
446159,lowE(2)/Clear-BBG75,12/31 22:00:00,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.004310,12
446160,lowE(2)/Clear-BBG75,12/31 23:00:00,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.053998,12


In [34]:
# Group by scenario and date/time for export to graphing
MonthlyData = allMonthlyData2.groupby(["Scenario", "Date_Time"])

MonthlyData.sum()

FanEnergy(kWh)  HeatingEnergy(therms)  \
Scenario           Date_Time                                                 
Clear/Clear         01/01  01:00:00        0.170103               0.176566   
                    01/01  02:00:00        0.185838               0.192898   
                    01/01  03:00:00        0.198987               0.206548   
                    01/01  04:00:00        0.211600               0.219640   
                    01/01  05:00:00        0.217641               0.225910   
...                                             ...                    ...   
lowE(2)/Clear-VB75  12/31  20:00:00        0.000000               0.000000   
                    12/31  21:00:00        0.000000               0.000000   
                    12/31  22:00:00        0.000000               0.000000   
                    12/31  23:00:00        0.035649               0.037004   
                    12/31  24:00:00        0.065570               0.068062   

                                     CoolingEnergy(kWh)  FanEnergy($)  \
Scenario           Date_Time                                            
Clear/Clear         01/01  01:00:00                 0.0      0.019052   
                    01/01  02:00:00                 0.0      0.020814   
                    01/01  03:00:00                 0.0      0.022287   
                    01/01  04:00:00                 0.0      0.023699   
                    01/01  05:00:00                 0.0      0.024376   
...                                                 ...           ...   
lowE(2)/Clear-VB75  12/31  20:00:00                 0.0      0.000000   
                    12/31  21:00:00                 0.0      0.000000   
                    12/31  22:00:00                 0.0      0.000000   
                    12/31  23:00:00                 0.0      0.003993   
                    12/31  24:00:00                 0.0      0.007344   

                                     HeatingEnergy($)  CoolingEnergy($)  \
Scenario           Date_Time                                              
Clear/Clear         01/01  01:00:00          0.195988               0.0   
                    01/01  02:00:00          0.214117               0.0   
                    01/01  03:00:00          0.229268               0.0   
                    01/01  04:00:00          0.243800               0.0   
                    01/01  05:00:00          0.250760               0.0   
...                                               ...               ...   
lowE(2)/Clear-VB75  12/31  20:00:00          0.000000               0.0   
                    12/31  21:00:00          0.000000               0.0   
                    12/31  22:00:00          0.000000               0.0   
                    12/31  23:00:00          0.041074               0.0   
                    12/31  24:00:00          0.075549               0.0   

                                     MonthlyCost($)  
Scenario           Date_Time                         
Clear/Clear         01/01  01:00:00        0.215039  
                    01/01  02:00:00        0.234931  
                    01/01  03:00:00        0.251555  
                    01/01  04:00:00        0.267499  
                    01/01  05:00:00        0.275136  
...                                             ...  
lowE(2)/Clear-VB75  12/31  20:00:00        0.000000  
                    12/31  21:00:00        0.000000  
                    12/31  22:00:00        0.000000  
                    12/31  23:00:00        0.045067  
                    12/31  24:00:00        0.082892  

[429240 rows x 7 columns]

In [35]:
# Export grouped file to csv (used before grouped by month)
MonthlyData.sum().round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

In [36]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyData.sum().round(2).to_json("data/final/hou_monthly_data.json")

In [37]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData2.groupby(["Scenario", "Month"])

MonthlyData2.sum()

FanEnergy(kWh)  HeatingEnergy(therms)  \
Scenario           Month                                          
Clear/Clear         01         69.937267              59.404715   
                    02         67.686261              43.585011   
                    03         73.453014              22.428684   
                    04         99.462352               4.085646   
                    05        163.668921               0.113253   
...                                  ...                    ...   
lowE(2)/Clear-VB75  08        201.909798               0.000000   
                    09        161.218852               0.000000   
                    10         63.375280               5.286591   
                    11         41.444003              17.957077   
                    12         52.077134              48.337583   

                          CoolingEnergy(kWh)  FanEnergy($)  HeatingEnergy($)  \
Scenario           Month                                                       
Clear/Clear         01             71.567127      7.832974         65.939234   
                    02            148.870250      7.580861         48.379362   
                    03            303.162769      8.226738         24.895839   
                    04            581.289313     11.139783          4.535067   
                    05           1022.627820     18.330919          0.125711   
...                                      ...           ...               ...   
lowE(2)/Clear-VB75  08           1327.866995     22.613897          0.000000   
                    09           1039.683724     18.056511          0.000000   
                    10            362.099479      7.098031          5.868116   
                    11            147.060659      4.641728         19.932355   
                    12             33.162382      5.832639         53.654717   

                          CoolingEnergy($)  MonthlyCost($)  
Scenario           Month                                    
Clear/Clear         01            8.015518       81.787726  
                    02           16.673468       72.633691  
                    03           33.954230       67.076807  
                    04           65.104403       80.779253  
                    05          114.534316      132.990946  
...                                    ...             ...  
lowE(2)/Clear-VB75  08          148.721103      171.335001  
                    09          116.444577      134.501088  
                    10           40.555142       53.521289  
                    11           16.470794       41.044878  
                    12            3.714187       63.201543  

[588 rows x 7 columns]

In [38]:
# Export grouped file to csv
MonthlyData2.sum().round(2).to_csv("../Scraper_Output/hou_monthly2_data.csv", header=True, index=True)

## Get monthly average illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [39]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]
Illuminance

,Scenario,Date_Time,Illuminance(lux),Month
0,Clear/Clear,01/01 01:00:00,0.0,01
1,Clear/Clear,01/01 02:00:00,0.0,01
2,Clear/Clear,01/01 03:00:00,0.0,01
3,Clear/Clear,01/01 04:00:00,0.0,01
4,Clear/Clear,01/01 05:00:00,0.0,01
...,...,...,...,...
446157,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,12
446158,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,12
446159,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,12
446160,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,12


In [40]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Date_Time"])

MonthlyIlluminance.mean()

Illuminance(lux)
Scenario           Date_Time                         
Clear/Clear         01/01  01:00:00               0.0
                    01/01  02:00:00               0.0
                    01/01  03:00:00               0.0
                    01/01  04:00:00               0.0
                    01/01  05:00:00               0.0
...                                               ...
lowE(2)/Clear-VB75  12/31  20:00:00               0.0
                    12/31  21:00:00               0.0
                    12/31  22:00:00               0.0
                    12/31  23:00:00               0.0
                    12/31  24:00:00               0.0

[429240 rows x 1 columns]

In [41]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

In [42]:
# # Export grouped file to json, issues with exporting grouped csv
# MonthlyIlluminance.mean().round(2).to_json("data/final/hou_monthly_illuminance.json")

In [43]:
# Group by scenario
MonthlyIlluminance2 = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance2.mean()

Illuminance(lux)
Scenario           Month                  
Clear/Clear         01         3732.829463
                    02         4303.780642
                    03         1074.314935
                    04         1092.766152
                    05          934.979390
...                                    ...
lowE(2)/Clear-VB75  08           25.472227
                    09           33.326059
                    10           36.690200
                    11           32.555067
                    12           28.279800

[588 rows x 1 columns]

In [44]:
# Export grouped file to csv
MonthlyIlluminance2.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance2.csv", header=True, index=True)

------

## Get hourly data

In [45]:
all_data3 = all_data1.copy(deep=True)

#all_data3

In [46]:
# Get wanted columns
# Include hourly illumanc
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [47]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [48]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly["FanEnergy($)"] = (get_hourly["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly["HeatingEnergy($)"] = ((get_hourly["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly["CoolingEnergy($)"] = (get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_hourly

In [49]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


------

## Get hourly cost data

In [50]:
get_hourly_costs = all_data3.copy(deep=True)

# get_hourly_costs

In [51]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [52]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)


## Combine hourly energy and cost data

In [53]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner" )

allHourlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.0,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...
446157,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
446158,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
446159,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.000382,0.003929,0.0
446160,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.004784,0.049214,0.0


In [54]:
# Export grouped file to csv for daily cost sums
# allHourlyData.to_csv("../Scraper_Output/hou_daily_cost_sum.csv", header=True, index=False)